In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from math import ceil
import time
import os
import win32com.client

headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.41 YaBrowser/21.2.0.2458 Yowser/2.5 Safari/537.36'
}

#### Этап 1. Собираем ссылки на дела

In [2]:
def get_n_pages_MGS(article, year):
    article = str(article)
    if ', ' in article:
        article = article.replace(', ', '%2C+')
    first_page_url = f'https://mos-gorsud.ru/search?formType=fullForm&courtAlias=&uid=&instance=1&processType=6&letterNumber=&caseNumber=&participant=&codex={article}&judge=&publishingState=&documentType=ab1da71&documentText=&year={year}&caseDateFrom=&caseDateTo=&caseFinalDateFrom=&caseFinalDateTo=&caseLegalForceDateFrom=&caseLegalForceDateTo=&docsDateFrom=&docsDateTo=&documentStatus=2&page=1'
    global headers
    response = requests.get(first_page_url, headers=headers)
    soup = BeautifulSoup(response.text)
    try:
        n_cases = int(soup.find('div', {'class':'resultsearch_text'}).get_text().strip().split('\n')[0].split(': ')[1])
    except:
        n_cases = 0
    n_pages = ceil(n_cases/15)
    return n_pages

In [3]:
def build_url(article, year, page=1):
    article = str(article)
    if ', ' in article:
        article = article.replace(', ', '%2C+')
    url = f'https://mos-gorsud.ru/search?formType=fullForm&courtAlias=&uid=&instance=1&processType=6&letterNumber=&caseNumber=&participant=&codex={article}&judge=&publishingState=&documentType=ab1da71&documentText=&year={year}&caseDateFrom=&caseDateTo=&caseFinalDateFrom=&caseFinalDateTo=&caseLegalForceDateFrom=&caseLegalForceDateTo=&docsDateFrom=&docsDateTo=&documentStatus=2&page={page}'
    return url

In [4]:
def parse_one_page_MGS(url):
    global headers
    response = requests.get(url, headers=headers)
    table = pd.read_html(response.text)[0]
    soup = BeautifulSoup(response.text)
    nobrs = [tag for tag in soup.find('tbody').find_all('nobr') if tag.find('a') is not None]
    urls = ['https://mos-gorsud.ru' + tag.a.get('href').split('?')[0] for tag in nobrs]
    table['URL'] = urls
    return table

In [5]:
articles = ['105', '107', '111, ч.4']
year = 2019
results = pd.DataFrame()

for article in articles:
    n_pages = get_n_pages_MGS(article, year)
    
    for page in range(1, n_pages+1):
        print(f'Статья {article}, год {year}, страница {page}/{n_pages}')
        url = build_url(article, year, page)
        page_results = parse_one_page_MGS(url)
        results = pd.concat([results, page_results])
results = results.reset_index(drop=True)
results.head()

Статья 105, год 2019, страница 1/10
Статья 105, год 2019, страница 2/10
Статья 105, год 2019, страница 3/10
Статья 105, год 2019, страница 4/10
Статья 105, год 2019, страница 5/10
Статья 105, год 2019, страница 6/10
Статья 105, год 2019, страница 7/10
Статья 105, год 2019, страница 8/10
Статья 105, год 2019, страница 9/10
Статья 105, год 2019, страница 10/10
Статья 111, ч.4, год 2019, страница 1/6
Статья 111, ч.4, год 2019, страница 2/6
Статья 111, ч.4, год 2019, страница 3/6
Статья 111, ч.4, год 2019, страница 4/6
Статья 111, ч.4, год 2019, страница 5/6
Статья 111, ч.4, год 2019, страница 6/6


,Номер дела ~ материала,Стороны,Текущее состояние,Судья,Статья,Категория дела,Список дел,URL
0,01-0770/2019,"Подсудимый: Рахматов Х.З. (Ст. 105, Ч. 1;)","Вступило в силу, 28.01.2020",Борисенкова Н.В.,NaN,NaN,Уголовные дела первой инстанции (Нагатинский р...,https://mos-gorsud.ru/rs/nagatinskij/services/...
1,01-0707/2019,"Подсудимый: Бредихин Д.С. (Ст. 105, Ч. 1;)","Вступило в силу, 04.03.2020",Борисенкова Н.В.,NaN,NaN,Уголовные дела первой инстанции (Нагатинский р...,https://mos-gorsud.ru/rs/nagatinskij/services/...
2,01-0654/2019,"Подсудимый: Палаткин В.В. (Ст. 162, Ч. 4, п. ...","Вступило в силу, 09.10.2019",Смирнова М.М.,NaN,NaN,Уголовные дела первой инстанции (Перовский рай...,https://mos-gorsud.ru/rs/perovskij/services/ca...
3,01-0643/2019,"Подсудимый: Федосеев А.В. (Ст. 105, Ч. 1;)","Изменено, 11.03.2020",Маслов А.В.,NaN,NaN,Уголовные дела первой инстанции (Люблинский ра...,https://mos-gorsud.ru/rs/lyublinskij/services/...
4,01-0621/2019,"Подсудимый: Ираймжанов Э.И. (Ст. 105, Ч. 1;)","Вступило в силу, 31.12.2019",Бахвалова Л.А.,NaN,NaN,Уголовные дела первой инстанции (Измайловский ...,https://mos-gorsud.ru/rs/izmajlovskij/services...


#### Этап 2. Добавляем ссылки на тексты приговоров

In [6]:
def get_verdict_url(case_url):
    global headers
    case_url += '#tabs-3'
    response = requests.get(case_url, headers=headers)
    soup = BeautifulSoup(response.text)
    tags_with_files = [tag for tag in soup.find_all('tbody')[-1].find_all('tr') if tag.find('a') is not None]
    verdict_file = [tag.a.get('href') for tag in tags_with_files if 'Приговор' in tag.text][0]
    verdict_file = 'https://mos-gorsud.ru' + verdict_file
    return verdict_file

In [7]:
n = len(results)
for i in results.index:
    if i==0 or (i+1)%25==0:
        print(f'{i+1}/{n}')
    case_url = results.loc[i, 'URL']
    try:
        results.loc[i, 'URL_verdict'] = get_verdict_url(case_url)
    except:
        results.loc[i, 'URL_verdict'] = None
    time.sleep(0.3)

1/227
25/227
50/227
75/227
100/227
125/227
150/227
175/227


KeyboardInterrupt: 

In [8]:
results.to_excel(f'MOSGORSUD_urls_{year}.xlsx')

#### Этап 3. Добавляем тексты приговоров

In [10]:
year = 2011
results = pd.read_excel(f'MOSGORSUD_urls_{year}.xlsx', index_col=0)
#results['custom_id'] = results['Номер дела ~ материала'].apply(lambda x: x.split('(')[0])# + ' ' + results['Стороны'].apply(lambda x: x.replace('Подсудимый: ', '').split('(')[0].strip().split(' ')[0])
results['custom_id'] = results['URL'].apply(lambda x: x.split('/')[-1])
results

,Номер дела ~ материала,Стороны,Текущее состояние,Судья,Статья,Категория дела,Список дел,URL,URL_verdict,custom_id
0,2-0133/2011,Подсудимый: Ахмедов Р.И.О. (Ст. 116 (отм. 03....,"Вступило в силу, 18.04.2012",Вырышева И.В.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/5...,70f5cdde-8088-4f3d-b04d-eae1cff2ae90
1,2-0132/2011,"Подсудимый: Ртищев П.В. (Ст. 105, Ч. 2, пп. ж...","Вступило в силу, 05.04.2012",Мелехин П.В.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/6...,2f48e67d-23aa-4ab3-94ca-0ec5ed6d61f5
2,2-0118/2011,Подсудимый: Айдаев А.Л. (Ст. 116 (отм. 03.07....,"Изменено, 19.01.2012",Коротков А.Г.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/7...,6583dab0-2ce9-46f5-90a6-fce21d4eb774
3,2-0117/2011,"Подсудимый: Абдусаматов С.А. (Ст. 162, Ч. 4, ...","Вступило в силу, 26.12.2011",Новиков В.А.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/d...,f3dc99da-6d35-4fab-808d-f46f0155b632
4,2-0115/2011,"Подсудимый: Байматов А.А. (Ст. 105, Ч. 2, п. ...","Вступило в силу, 17.11.2011",NaN,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/c...,d804abbf-3a90-4f2f-8592-0e3d3fa71c28
5,2-0111/2011,"Подсудимый: Елин В.В. (Ст. 209, Ч. 2;; Ст. 22...","Вступило в силу, 29.02.2012",Мелехин П.В.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/c...,ae037856-0327-44ae-b0a7-3ec59f04e5c5
6,2-0105/2011,"Подсудимый: Кучмезов К.К. (Ст. 162, Ч. 4, пп....","Вступило в силу, 11.01.2012",Смолкина Л.М.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/a...,554c15da-6cd0-47b7-9068-d1bd354b0413
7,2-0094/2011,"Подсудимый: Веденин М.В. (Ст. 162, Ч. 2 (отм....","Вступило в силу, 09.11.2011",Новиков В.А.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/7...,57122096-7e3e-4fd0-ad69-265732e4d90d
8,2-0091/2011,"Подсудимый: Сироткин Н.А. (Ст. 162, Ч. 4, п. ...","Изменено, 05.09.2011",NaN,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/5...,a8d8468d-a5b0-499a-a76e-df2eb7768f98
9,2-0090/2011,"Подсудимый: Самуйлов И.В. (Ст. 105, Ч. 2, п. ...","Изменено, 08.09.2011",Николенко Л.И.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/a...,38441820-4ee4-486e-ab45-3c209d62bfac


In [11]:
def get_text_by_url_MGS(url, verdict_id, folder='MOSGORSUD_CASES'):
    global headers
    filename = f'{verdict_id}.doc'
    if folder not in os.listdir():
        os.mkdir(folder)
    if filename not in os.listdir(folder):    
        response = requests.get(url, headers=headers)
        with open(folder + "\\" + filename, 'wb') as file:
            file.write(response.content)
    word = win32com.client.Dispatch("Word.Application")
    word.visible = False
    wb = word.Documents.Open(os.getcwd() + "\\" + folder + "\\" + filename)
    doc = word.ActiveDocument
    text = doc.Range().Text
    return text

In [12]:
for i in results.index:
    if i==0 or i%25==0:
        print(f'{i+1}/{len(results)}')
    verdict_url = results.loc[i, 'URL_verdict']
    verdict_id = results.loc[i, 'custom_id']
    #try:
    results.loc[i, 'text'] = get_text_by_url_MGS(verdict_url, verdict_id)
#     except:
#         results.loc[i, 'text'] = None

1/36
26/36


In [13]:
results

,Номер дела ~ материала,Стороны,Текущее состояние,Судья,Статья,Категория дела,Список дел,URL,URL_verdict,custom_id,text
0,2-0133/2011,Подсудимый: Ахмедов Р.И.О. (Ст. 116 (отм. 03....,"Вступило в силу, 18.04.2012",Вырышева И.В.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/5...,70f5cdde-8088-4f3d-b04d-eae1cff2ae90,\r П Р И Г О В О Р\rИменем Российской Федерац...
1,2-0132/2011,"Подсудимый: Ртищев П.В. (Ст. 105, Ч. 2, пп. ж...","Вступило в силу, 05.04.2012",Мелехин П.В.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/6...,2f48e67d-23aa-4ab3-94ca-0ec5ed6d61f5,ПРИГОВОР \rИменем Российской Федерации \r\r\r...
2,2-0118/2011,Подсудимый: Айдаев А.Л. (Ст. 116 (отм. 03.07....,"Изменено, 19.01.2012",Коротков А.Г.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/7...,6583dab0-2ce9-46f5-90a6-fce21d4eb774,\rПРИГОВОР\rИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ\r\rг. ...
3,2-0117/2011,"Подсудимый: Абдусаматов С.А. (Ст. 162, Ч. 4, ...","Вступило в силу, 26.12.2011",Новиков В.А.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/d...,f3dc99da-6d35-4fab-808d-f46f0155b632,\t\t\t\t\t\t\t\t\t Дело №2-11...
4,2-0115/2011,"Подсудимый: Байматов А.А. (Ст. 105, Ч. 2, п. ...","Вступило в силу, 17.11.2011",NaN,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/c...,d804abbf-3a90-4f2f-8592-0e3d3fa71c28,№ 2-115/ 11\r ...
5,2-0111/2011,"Подсудимый: Елин В.В. (Ст. 209, Ч. 2;; Ст. 22...","Вступило в силу, 29.02.2012",Мелехин П.В.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/c...,ae037856-0327-44ae-b0a7-3ec59f04e5c5,\rПРИГОВОР \rИменем Российской Федерации \r\r\...
6,2-0105/2011,"Подсудимый: Кучмезов К.К. (Ст. 162, Ч. 4, пп....","Вступило в силу, 11.01.2012",Смолкина Л.М.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/a...,554c15da-6cd0-47b7-9068-d1bd354b0413,\r\rПРИГОВОР\rИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ\r\rг...
7,2-0094/2011,"Подсудимый: Веденин М.В. (Ст. 162, Ч. 2 (отм....","Вступило в силу, 09.11.2011",Новиков В.А.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/7...,57122096-7e3e-4fd0-ad69-265732e4d90d,\r \t\t\t\t\t\t\t\t\t Дело №2-9...
8,2-0091/2011,"Подсудимый: Сироткин Н.А. (Ст. 162, Ч. 4, п. ...","Изменено, 05.09.2011",NaN,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/5...,a8d8468d-a5b0-499a-a76e-df2eb7768f98,№ 2-91/ 11\r ...
9,2-0090/2011,"Подсудимый: Самуйлов И.В. (Ст. 105, Ч. 2, п. ...","Изменено, 08.09.2011",Николенко Л.И.,NaN,NaN,Уголовные дела первой инстанции (Московский го...,https://mos-gorsud.ru/mgs/services/cases/first...,https://mos-gorsud.ru/mgs/cases/docs/content/a...,38441820-4ee4-486e-ab45-3c209d62bfac,...


In [14]:
results['text'] = results['text'].str.replace('\r', ' ')
results.to_excel(f'MOSGORSUD_texts_{year}.xlsx')